<a href="https://colab.research.google.com/github/zhenyiqi/nanoGPT/blob/master/gpt_dev_zhenyiqi_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [24]:
import torch

In [26]:
from torch import nn

# Prepare dataset

## Download

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-01-31 00:11:40--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2024-01-31 00:11:40 (20.3 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r') as f:
  text = f.read()

In [3]:
print(f"length of the dataset is {len(text)}")

length of the dataset is 1115394


## encode characters into numbers (indices)

In [4]:
chars = sorted(list(set(text)))

In [5]:
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

In [7]:
encode = lambda s: [stoi[c] for c in s]
decode = lambda i: ''.join([itos[ind] for ind in i])

In [8]:
print(encode("hello there"))
print(decode(encode("hello there")))

[46, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43]
hello there


In [9]:
import torch

In [11]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


## Create train/validation sets

In [12]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [13]:
# notation
block_size = 8 # this is the maximum length of the chunk (sequence length, in other places, this is also donated as T)
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [14]:
x = train_data[:block_size] # [18, 47, 56, 57, 58,  1, 15, 47]
y = train_data[1:block_size + 1] # [47, 56, 57, 58,  1, 15, 47, 58]
for t in range(block_size):
  context = x[:t + 1]
  target = y[t]
  print(f"when input is {context}, the target is : {target}")


when input is tensor([18]), the target is : 47
when input is tensor([18, 47]), the target is : 56
when input is tensor([18, 47, 56]), the target is : 57
when input is tensor([18, 47, 56, 57]), the target is : 58
when input is tensor([18, 47, 56, 57, 58]), the target is : 1
when input is tensor([18, 47, 56, 57, 58,  1]), the target is : 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is : 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is : 58


In [15]:
# generalize with extra batch size dimension
torch.manual_seed(1337)
batch_size = 4
block_size = 8
def get_batch(split):
  data = train_data if split == 'train' else val_data
  # a random vector of length batch_size, with its value sampled from
  # the range (0, len(data) - block_size), which is all possible values for the
  # the start of a sampled sequence.
  ix = torch.randint(len(data) - block_size, (batch_size, ))
  # now get the samples out, stacking them together
  x = torch.stack([data[i:i+block_size] for i in ix])  # (block_size, batch_size)
  y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # (block_size, batch_size)
  return x, y

In [16]:
xb, yb = get_batch('train')

In [17]:
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print('----')

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----


In [18]:
for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"when input is {context.tolist()}, the target is {target}")

when input is [24], the target is 43
when input is [24, 43], the target is 58
when input is [24, 43, 58], the target is 5
when input is [24, 43, 58, 5], the target is 57
when input is [24, 43, 58, 5, 57], the target is 1
when input is [24, 43, 58, 5, 57, 1], the target is 46
when input is [24, 43, 58, 5, 57, 1, 46], the target is 43
when input is [24, 43, 58, 5, 57, 1, 46, 43], the target is 39
when input is [44], the target is 53
when input is [44, 53], the target is 56
when input is [44, 53, 56], the target is 1
when input is [44, 53, 56, 1], the target is 58
when input is [44, 53, 56, 1, 58], the target is 46
when input is [44, 53, 56, 1, 58, 46], the target is 39
when input is [44, 53, 56, 1, 58, 46, 39], the target is 58
when input is [44, 53, 56, 1, 58, 46, 39, 58], the target is 1
when input is [52], the target is 58
when input is [52, 58], the target is 1
when input is [52, 58, 1], the target is 58
when input is [52, 58, 1, 58], the target is 46
when input is [52, 58, 1, 58, 46

In [19]:
xb

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

# Modeling

### LayerNorm

In [20]:
dropout = 0.2

In [21]:
# class LayerNorm(nn.Module):
#   def __init__(self, dim, eps=1e-5):
#     self.eps = eps
      # gamma and beta are trainable variables
#     self.gamma = torch.ones(dim)
#     self.beta = torch.ones(dim)

#   def forward(self, x):
#     xmean = x.mean(1, keepdim=True)
#     xvar = x.var(1, keepdim=True)
#     xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
#     out = self.gamma * xhat + self.beta
#     return out

#   def parameters(self):
#     return [self.gamma, self.beta]

### self-attention head with dropout

In [27]:
class SelfAttentionHead(nn.Module):
  def __init__(self, head_size):
    """self-attention head.

    there will be multiple heads. head_size * n_head = n_embed.
    """
    super().__init__()
    self.key = nn.Linear(n_embed, head_size, bias=False)
    self.query = nn.Linear(n_embed, head_size, bias=False)
    self.value = nn.Linear(n_embed, head_size, bias=False)
    # tril is not the parameter of the module, so we call it a buffer
    # (not a parameter)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B, T, C = x.shape
    k = self.key(x) # (B, T, head_size)
    q = self.query(x) # (B, T, head_size)

    wei = q @ k.transpose(-2, -1) * C ** -0.5# (B, T, T)
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
    wei = F.softmax(wei, dim=-1) # (B, T, T)
    wei = self.dropout(wei)

    v = self.value(x) # (B, T, head_size)
    out = wei @ v # (B, T, head_size)
    return out

### Multi-head attention with a projection

In [28]:
class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([SelfAttentionHead(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embed, n_embed)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    attention_out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.proj(attention_out)
    return out

### FeedForward with dropout

In [29]:
class FeedForward(nn.Module):
  def __init__(self, n_embed):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embed, n_embed * 4),
        nn.ReLU(),
        nn.Linear(n_embed * 4, n_embed),
        nn.Dropout(dropout)
    )

  def forward(self, x):
    return self.net(x)

### Block of (self-attention and feed-forward)

In [30]:
class Block(nn.Module):
  def __init__(self, n_embed, n_head):
    super().__init__()
    head_size = n_embed // n_head
    self.sa = MultiHeadAttention(n_head, head_size=head_size)
    self.ffwd = FeedForward(n_embed)
    self.ln1 = nn.LayerNorm(n_embed)
    self.ln2 = nn.LayerNorm(n_embed)

  def forward(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x

### Model with multiple layers of (self-attention, feed-forward) and layernorm + skipadd

In [31]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

n_embed = 32
n_head = 4
n_layer = 3

class GPT(nn.Module):
  def __init__(self):
    super().__init__()
    # an embedding is a table that maps each token to a vector
    # nn.Embedding(n1, n2) maps a token whose value is up to n1 (starting from 0)
    # to a vector of size n2. Here we map each token to a vector of size vocab_size
    # (One example is one-hot encoding)
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
    self.position_embedding_table = nn.Embedding(block_size, n_embed)
    self.blocks = nn.Sequential(*[Block(n_embed, n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embed) # final layer norm
    # This is the output layer that maps the transformed embeddings back to the
    # original vocab_size vector - logits.
    self.lm_head = nn.Linear(n_embed, vocab_size)

  def forward(self, idx, targets=None):
    """A Forward proporgation.

    idx: the input. Dimension [B, T].
    targets: target. Dimension [B, T], [B, i]th element is the target of sequence
      idx[B, :i].
    """
    B, T = idx.shape

    # idx and targets are both (B, T), meaning (batch_size, block_size)
    token_embed = self.token_embedding_table(idx) # (B, T, C = n_embed)
    # pos_embed = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
    pos_embed = self.position_embedding_table(torch.arange(T)) # (T, C)

    x = token_embed + pos_embed
    x = self.blocks(x)
    x = self.ln_f(x)
    logits = self.lm_head(x) # (B, T, C = vocab_size)
    # here is a bit tricky, please refer to
    # https://pytorch.org/docs/stable/generated/torch.nn.functional.cross_entropy.html
    # to understand what's in there
    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B * T, C)
      targets = targets.view(B * T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    """Generate the next token for max_new_tokens of times for the batch idx.

    idx: a batch of samples, of size (B, T).
    max_new_tokens: int, how many tokens to be generated given a sample.
    """
    for _ in range(max_new_tokens):
      # crop the input size so that it's at most the same as block_size when
      # generating
      idx_cond = idx[:, -block_size:]
      # get the predictions
      logits, loss = self(idx_cond) # logits: (B, T, C)
      # print(logits.shape)
      # Last one of the sequence
      logits = logits[:, -1, :] # (B, C)
      probs = F.softmax(logits, dim=-1) # (B, C)
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      idx = torch.cat((idx, idx_next), dim=1) # (B, T + 1)
    return idx

model = GPT()
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.2523, grad_fn=<NllLossBackward0>)


In [32]:
sum(p.numel() for p in model.parameters())

42369

# Training

In [33]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [36]:
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'

eval_iters = 200

In [37]:
# To tell Pytorch that everything inside this function will not be used in
# loss.backward() so that it won't allocate any memory for the local variables
# for backproporgation.
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [39]:
batch_size = 32
for iter in range(max_iters):
  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  # sample a batch of data
  xb, yb = get_batch('train')

  # evaluate the loss
  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

step 0: train loss 4.3104, val loss 4.3100
step 500: train loss 2.4657, val loss 2.4560
step 1000: train loss 2.3117, val loss 2.3276
step 1500: train loss 2.2432, val loss 2.2604
step 2000: train loss 2.1860, val loss 2.2131
step 2500: train loss 2.1573, val loss 2.1901
step 3000: train loss 2.1427, val loss 2.1602
step 3500: train loss 2.1033, val loss 2.1457
step 4000: train loss 2.0895, val loss 2.1274
step 4500: train loss 2.0572, val loss 2.0928


# Eval

In [40]:
idx = torch.zeros((1, 1), dtype=torch.long)

print(decode(model.generate(idx, max_new_tokens=300)[0].tolist()))


MEONTUCES:
Leash for hous?
Taire tome and at andients for perecaint:
Agnes notes'd me; ou me staed thoGood pettrane:
I do meng
Fich'd sownerreawirs that she aman: le golk a ne yough, gukit, mane mopinair!
Arve; the loot.

I wethyit? pare,
The pliivery of not oun, cony the him wo af that's of mittees


# Save the checkpoint

In [41]:
! ls

input.txt  sample_data


In [42]:
output_dir = "./checkpoints"